In [67]:
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
import torch
import numpy as np

In [42]:
# 分割数据集（无用）
def data_split(data, test_size):
    trn_data = {}
    val_data = {}

    for label, data in data.items():
        lens = len(data[label])
        print(lens)
        trn_data[label] = data[: lens * (1 - test_size)]
        val_data[label] = data[lens * (1 - test_size) :]
    
    return trn_data, val_data

In [62]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [66]:
# 读取mat
trn_mat = scipy.io.loadmat("data/train_data.mat")
trn_data = trn_mat["train"]
# print(len(trn_data))
x_len = len(trn_data)

# 标注数据label
trn_label = [i + 1 for i in range(x_len)]
# print(trn_label)

# 将label与data组合为字典
trn_dict = {}
for i in range(x_len):
    trn_dict[i+1] = torch.tensor(trn_data[i])
# print(len(trn_dict))
print(type(trn_dict[1]))

# 读取测试集
val_mat = scipy.io.loadmat('data/test_data.mat')
val_data = torch.tensor(val_mat['test'])
val_data = val_data
print(type(val_data[1]))
print(len(val_data))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
1000


In [68]:
# 定义SVM
class LinearSVM:
    # 定义学习率、正则化强度、迭代次数
    def __init__(self, lr=1e-3, reg=1e-5, num_iter=100):
        self.lr = lr
        self.reg = reg
        self.num_iter = num_iter
        
    # 计算损失和梯度
    def compute_loss_and_grad(self, x, y):
        num_trn = x.shape[0]
        num_cls = np.max(y) # 样本类别的数量，在预处理数据时已+1，此处无需+1
        scores = x.dot(self.W)  # 获取分数矩阵
        correct_cls_scores = scores[np.arange(num_trn), y]  # 
        
        margins = np.maximum(0, scores - correct_cls_scores[:, np.newaxis] + 1)
        margins[np.arange(num_trn), y] = 0
        loss = np.sum(margins) / num_trn
        loss += 0.5 * self.reg * np.sum(self.W * self.W)
        
        margins[margins>0] = 1
        margins[np.arange(num_trn), y] = -np.sum(margins, axis=1)
        dW = x.T.dot(margins) / num_trn
        dW += self.reg * self.W
        
        return loss, dW

    # 训练
    def fit(self, x, y):
        num_trn, dim = x.shape
        num_cls = np.max(y) + 1
        self.W = 0.001 * np.random.randn(dim, num_cls)

        for i in range(self.num_iter):
            loss, grad = self.compute_loss_and_grad(x, y)
            self.W -= self.lr * grad
            if i % 10 == 0:
                print(f"Iteration {i}/{self.num_iter}, Loss: {loss}")
                
    # 预测
    def predict(self, x):
        scores = x.dot(self.W)
        return x.argmax(scores, axis=1)